In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv('german.csv', sep=';')

X = data.iloc[:, 1:].to_numpy()
"""
X = data[
    [
    'Account_Balance',
    'Duration_of_Credit_monthly',
    'Payment_Status_of_Previous_Credit',
    'Length_of_current_employment',
    'Sex_Marital_Status',
    'Guarantors',
    'Most_valuable_available_asset',
    'No_of_Credits_at_this_Bank',
    ]
].to_numpy()
"""
y = data.iloc[:, 0].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
data.head()

,Creditability,Account_Balance,Duration_of_Credit_monthly,Payment_Status_of_Previous_Credit,Purpose,Credit_Amount,Value_Savings_Stocks,Length_of_current_employment,Instalment_per_cent,Sex_Marital_Status,...,Duration_in_Current_address,Most_valuable_available_asset,Age_years,Concurrent_Credits,Type_of_apartment,No_of_Credits_at_this_Bank,Occupation,No_of_dependents,Telephone,Foreign_Worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,...,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,...,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,...,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,...,4,2,38,1,2,2,2,1,1,2


In [8]:
# Задаем список возможных значений гиперпараметров
param_grid = {
    'loss': ['log_loss', 'exponential'],
    'learning_rate': [10**(-i) for i in range(0, 3)], #[0.1, 0.01, 0.001],
    'n_estimators': [i for i in range(100, 400+1, 50)],
    #'subsample': [1.0],
    'criterion': ['friedman_mse', 'squared_error'],
    'min_samples_split': [i for i in range(2, 10, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    #'min_weight_fraction_leaf': [0.0],
    #'max_depth': [3],
    #'min_impurity_decrease':[0.0],
    'random_state': [42]
}
"""
[
    'ccp_alpha', 
    'criterion', 
    'init', 
    'learning_rate', 
    'loss', 
    'max_depth', 
    'max_features', 
    'max_leaf_nodes', 
    'min_impurity_decrease', 
    'min_samples_leaf', 
    'min_samples_split', 
    'min_weight_fraction_leaf', 
    'n_estimators', 
    'n_iter_no_change', 
    'random_state', 
    'subsample', 
    'tol', 
    'validation_fraction', 
    'verbose', 
    'warm_start'
]
"""
# Создаем модель Gradient Boosting
gb_model = GradientBoostingClassifier()

# Инициализируем GridSearchCV для перебора гиперпараметров
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, scoring='roc_auc', cv=3)

# Обучение модели с перебором гиперпараметров
grid_search.fit(X_train, y_train)

# Лучшие найденные значения гиперпараметров
best_params = grid_search.best_params_

# Вывод промежуточных результатов
best_roc_auc = 0
par = {}
for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
    if mean_score >= best_roc_auc:
        best_roc_auc = mean_score
        par = params
        best_model = GradientBoostingClassifier(**params)
        best_model.fit(X_train, y_train)
        best_pred = best_model.predict_proba(X_test)[:, 1]
        print("Лучшие значения гиперпараметров:")
        print(params)
        print(f"ROC AUC: {best_roc_auc:.5f}")
        print("-------------------------------")
# Расчет метрик для лучшего найденного классификатора
best_roc_auc = roc_auc_score(y_test, best_pred)

"""
В этом коде мы используем модель Gradient Boosting и скользящее окно GridSearchCV 
для перебора заданных гиперпараметров `n_estimators` и `learning_rate`. 
Затем мы выводим промежуточные результаты, сохраняя лучшую модель и лучшее значение метрики ROC AUC, 
когда ROC AUC больше или равен предыдущему лучшему значению.
"""

print("\nЛучший классификатор метрики:")
print(f"ROC AUC: {best_roc_auc:.5f}")
print(par)

Лучшие значения гиперпараметров:
{'criterion': 'friedman_mse', 'learning_rate': 1, 'loss': 'log_loss', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}
ROC AUC: 0.74397
-------------------------------
Лучшие значения гиперпараметров:
{'criterion': 'friedman_mse', 'learning_rate': 1, 'loss': 'log_loss', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 150, 'random_state': 42}
ROC AUC: 0.74401
-------------------------------
Лучшие значения гиперпараметров:
{'criterion': 'friedman_mse', 'learning_rate': 1, 'loss': 'log_loss', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 200, 'random_state': 42}
ROC AUC: 0.74765
-------------------------------
Лучшие значения гиперпараметров:
{'criterion': 'friedman_mse', 'learning_rate': 1, 'loss': 'log_loss', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 400, 'random_state': 42}
ROC AUC: 0.74800
-------------------------------
Лучшие значения гиперпараметров:
{'crite

In [9]:
# Обучение Gradient Boosting
gb_model = GradientBoostingClassifier(**par)
gb_model.fit(X_train, y_train)

# Прогноз на тестовых данных
#gb_pred = gb_model.predict(X_test)
gb_pred = gb_model.predict_proba(X_test)[:,1]

# Расчет метрик для Gradient Boosting
gb_roc_auc = roc_auc_score(y_test, gb_pred)

print("\nGradient Boosting метрики:")
print(f"ROC AUC: {gb_roc_auc:.2f}")
print(f"ROC AUC: {gb_roc_auc:.5f}")


Gradient Boosting метрики:
ROC AUC: 0.78
ROC AUC: 0.77700
